In [1]:
from PIL import Image

from mtcnn.mtcnn import MTCNN
from keras.models import load_model
import numpy as np
from numpy import asarray
from numpy import expand_dims

import pickle
import cv2

In [2]:
detector = MTCNN()
model = load_model('facenet_keras.h5')

In [3]:
myfile = open("data.pkl", "rb")
database = pickle.load(myfile)
myfile.close()

In [5]:
cap = cv2.VideoCapture(0)

while(1):
    _, gbr1 = cap.read()
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)                  # konversi dari OpenCV ke PIL
    
    gbr_array = asarray(gbr)
    
    wajah = detector.detect_faces(gbr_array)
    
    if len(wajah)>0:
        x1, y1, width, height = wajah[0]['box']         
    else:
        x1, y1, width, height = 1, 1, 10, 10
        
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = gbr_array[y1:y2, x1:x2]                        
    
    face = Image.fromarray(face)                       
    face = face.resize((160,160))
    face = asarray(face)
    
    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    
    face = expand_dims(face, axis=0)
    signature = model.predict(face)
    
    min_dist=100
    identity=' '
    for key, value in database.items() :
        dist = np.linalg.norm(value-signature)
        if dist < min_dist:
            min_dist = dist
            identity = key
            
    cv2.putText(gbr1,identity, (100,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.rectangle(gbr1,(x1,y1),(x2,y2), (0,255,0), 2)
        
    cv2.imshow('res',gbr1)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()
cap.release()

In [10]:
cv2.destroyAllWindows()
cap.release()